In [45]:
import h5py
import numpy as np
import pandas as pd
import json
import yaml
from ludwig import LudwigModel
import copy
import ray

In [ ]:
#quick way to get base model specs
with open("titanic.yaml", 'r') as stream:
    a = yaml.load(stream)
a


In [46]:
input_features = [{'name': 'Pclass', 'type': 'category'},
  {'name': 'Sex', 'type': 'category'},
  {'name': 'Age',
   'type': 'numerical',
   'missing_value_strategy': 'fill_with_mean'},
  {'name': 'SibSp', 'type': 'numerical'},
  {'name': 'Parch', 'type': 'numerical'},
  {'name': 'Fare',
   'type': 'numerical',
   'missing_value_strategy': 'fill_with_mean'},
  {'name': 'Embarked', 'type': 'category', 'representation': 'sparse'}]

combiner = {'type': 'concat', 'num_fc_layers': 1, 'fc_size': 48}

output_features = [{'name': 'Survived', 'type': 'binary'}]

training = {'optimizer': {'type': 'adam', 'beta1': 0.9, 'beta2': 0.999, 'epsilon': 1e-08},
 'epochs': 100,
 'regularizer': 'l2',
 'regularization_lambda': 0,
 'learning_rate': 0.001,
 'batch_size': 128,
 'dropout_rate': 0.0,
 'early_stop': 5,
 'reduce_learning_rate_on_plateau': 0,
 'reduce_learning_rate_on_plateau_patience': 5,
 'reduce_learning_rate_on_plateau_rate': 0.5,
 'increase_batch_size_on_plateau': 0,
 'increase_batch_size_on_plateau_patience': 5,
 'increase_batch_size_on_plateau_rate': 2,
 'increase_batch_size_on_plateau_max': 512,
 'decay': False,
 'decay_steps': 10000,
 'decay_rate': 0.96,
 'staircase': False,
 'gradient_clipping': None,
 'validation_field': 'combined',
 'validation_measure': 'loss',
 'bucketing_field': None,
 'learning_rate_warmup_epochs': 5}

In [47]:
base_model = {'input_features': input_features, 'output_features': output_features, 'combiner': combiner, 'training': training}

In [49]:
f = open("titanic_full.yaml", "w")
f.write(yaml.dump(base_model))
f.close()

In [51]:
model = LudwigModel({'input_features': input_features, 'output_features': output_features, 'combiner': combiner, 'training': training})
train_stats = model.train(data_hdf5='titanic.hdf5', train_set_metadata_json='titanic.json')


{'combiner': {'fc_size': 48, 'num_fc_layers': 1, 'type': 'concat'},
 'input_features': [{'name': 'Pclass', 'type': 'category'},
  {'name': 'Sex', 'type': 'category'},
  {'missing_value_strategy': 'fill_with_mean',
   'name': 'Age',
   'type': 'numerical'},
  {'name': 'SibSp', 'type': 'numerical'},
  {'name': 'Parch', 'type': 'numerical'},
  {'missing_value_strategy': 'fill_with_mean',
   'name': 'Fare',
   'type': 'numerical'},
  {'name': 'Embarked', 'representation': 'sparse', 'type': 'category'}],
 'output_features': [{'name': 'Survived', 'type': 'binary'}],
 'training': {'batch_size': 128,
  'bucketing_field': None,
  'decay': False,
  'decay_rate': 0.96,
  'decay_steps': 10000,
  'dropout_rate': 0.0,
  'early_stop': 5,
  'epochs': 100,
  'gradient_clipping': None,
  'increase_batch_size_on_plateau': 0,
  'increase_batch_size_on_plateau_max': 512,
  'increase_batch_size_on_plateau_patience': 5,
  'increase_batch_size_on_plateau_rate': 2,
  'learning_rate': 0.001,
  'learning_rate_wa

In [52]:
model = LudwigModel({'input_features': input_features, 'output_features': output_features, 'combiner': combiner, 'training': training})
train_stats = model.train(data_hdf5='titanic.hdf5', train_set_metadata_json='titanic.json')


/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [41]:
model = LudwigModel({'input_features': input_features, 'output_features': output_features, 'combiner': combiner, 'training': training})
train_stats = model.train(data_hdf5='titanic.hdf5', train_set_metadata_json='titanic.json')




Instructions for updating:
Colocations handled automatically by placer.


/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


In [44]:
train_stats


{'train': OrderedDict([('Survived',
               OrderedDict([('loss',
                             [3.1847047351655506,
                              2.026768735855345,
                              1.5473910619342137,
                              1.4372893560500373,
                              1.243487560938275,
                              1.0303204006618925,
                              0.9660346984863282,
                              0.9459801810128349,
                              0.8594449966672867,
                              0.7966346377418154,
                              0.758392333984375,
                              0.7139286707318019,
                              0.6825730217827691,
                              0.6579385908823164,
                              0.6317809937492249,
                              0.6104406932043651,
                              0.5909345233251178,
                              0.5741014026460194,
                              

In [9]:
def merge_dict(dct, merge_dct):
    """ Recursive dict merge. Inspired by :meth:``dict.update()``, instead of
    updating only top-level keys, dict_merge recurses down into dicts nested
    to an arbitrary depth, updating keys. The ``merge_dct`` is merged into
    ``dct``.
    :param dct: dict onto which the merge is executed
    :param merge_dct: dct merged into dct
    :return: None
    """
    dct = copy.deepcopy(dct)
    for k, v in merge_dct.items():
        if (k in dct and isinstance(dct[k], dict)
                and isinstance(merge_dct[k], Mapping)):
            dct[k] = merge_dict(dct[k], merge_dct[k])
        else:
            dct[k] = merge_dct[k]
    return dct

In [38]:
#simple example....try different sizes of fc after combiner, and try different batch_size
BASE = '/Users/benmackenzie/projects/Teradata/ludwig/examples/hyperparameters/'

def train(model_def, config, reporter):
    combiner = model_def['combiner']
    combiner = merge_dict(combiner, {'num_fc_layers': config['num_fc_layers']})
    training = model_def['training']
    training = merge_dict(training, {'batch_size': config['batch_size']})
    new_model_def = {'input_features': model_def['input_features'], 
                 'output_features': model_def['output_features'], 
                 'combiner': combiner, 
                 'training': training}
    model = LudwigModel(new_model_def)
    
    train_stats = model.train(data_hdf5=BASE+'titanic.hdf5', train_set_metadata_json=BASE+'titanic.json')
    return reporter(mean_accuracy=0.8)


    


In [39]:
from ray.tune import register_trainable, grid_search, run_experiments

#ray.init()

grid_search_space = {
    'num_fc_layers': grid_search([1,2,3,4]),
    'batch_size': grid_search([4,8,16,32,64,128])
}

register_trainable('train', lambda cfg, rptr: train(base_model, cfg, rptr))
run_experiments({'my_experiment': { 
    'run': 'train', 
    'stop': {'mean_accuracy': 0.9},
    'config': grid_search_space}
    })

                

2019-03-13 14:53:01,517	INFO tune.py:135 -- Tip: to resume incomplete experiments, pass resume='prompt' or resume=True to run_experiments()
2019-03-13 14:53:01,518	INFO tune.py:145 -- Starting a new experiment.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0/0 GPUs
Unknown memory usage. Please run `pip install psutil` (or ray[debug]) to resolve)

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 1/4 CPUs, 0/0 GPUs
Unknown memory usage. Please run `pip install psutil` (or ray[debug]) to resolve)
Result logdir: /Users/benmackenzie/ray_results/my_experiment
Number of trials: 24 ({'RUNNING': 1, 'PENDING': 23})
PENDING trials:
 - train_1_batch_size=8,num_fc_layers=1:	PENDING
 - train_2_batch_size=16,num_fc_layers=1:	PENDING
 - train_3_batch_size=32,num_fc_layers=1:	PENDING
 - train_4_batch_size=64,num_fc_layers=1:	PENDING
 - train_5_batch_size=128,num_fc_layers=1:	PENDING
 - train_6_batch_size=4,num_fc_layers=2:	PENDING
 - train_7_batch_size=8,num_fc_layers=2:	PENDING
 - train_8_batch_size=16,num_fc_layers=2:	PENDING
 - train_9_batch_size=32,num_fc_layers=2:	PENDING
  ... 5 not shown
 - train_15_batch_size=32,num_fc_layers=3:	PENDING
 - train_16_batc

2019-03-13 14:53:12,352	ERROR trial_runner.py:426 -- Error processing event.
Traceback (most recent call last):
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/trial_runner.py", line 389, in _process_events
    result = self.trial_executor.fetch_result(trial)
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/ray_trial_executor.py", line 252, in fetch_result
    result = ray.get(trial_future[0])
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/worker.py", line 2288, in get
    raise value
ray.exceptions.RayTaskError: ray_worker (pid=14521, host=192.168.219.94)
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/trainable.py", line 150, in train
    result = self._train()
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/function_runner.py", line 133, in _train
    result = self._status_reporter._get_and_clear_stat

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/4 CPUs, 0/0 GPUs
Unknown memory usage. Please run `pip install psutil` (or ray[debug]) to resolve)
Result logdir: /Users/benmackenzie/ray_results/my_experiment
Number of trials: 24 ({'RUNNING': 3, 'ERROR': 1, 'PENDING': 20})
ERROR trials:
 - train_2_batch_size=16,num_fc_layers=1:	ERROR, 1 failures: /Users/benmackenzie/ray_results/my_experiment/train_2_batch_size=16,num_fc_layers=1_2019-03-13_14-53-01h6117lvj/error_2019-03-13_14-53-12.txt
PENDING trials:
 - train_4_batch_size=64,num_fc_layers=1:	PENDING
 - train_5_batch_size=128,num_fc_layers=1:	PENDING
 - train_6_batch_size=4,num_fc_layers=2:	PENDING
 - train_7_batch_size=8,num_fc_layers=2:	PENDING
 - train_8_batch_size=16,num_fc_layers=2:	PENDING
 - train_9_batch_size=32,num_fc_layers=2:	PENDING
 - train_10_batch_size=64,num_fc_layers=2:	PENDING
 - train_11_batch_size=128,num_fc_layers=2:	PENDING
  ... 4 not shown
 - train_16_batch_size=64,num_fc_layers=3:	PENDING
 -

2019-03-13 14:53:12,572	ERROR trial_runner.py:426 -- Error processing event.
Traceback (most recent call last):
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/trial_runner.py", line 389, in _process_events
    result = self.trial_executor.fetch_result(trial)
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/ray_trial_executor.py", line 252, in fetch_result
    result = ray.get(trial_future[0])
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/worker.py", line 2288, in get
    raise value
ray.exceptions.RayTaskError: ray_worker (pid=14518, host=192.168.219.94)
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/trainable.py", line 150, in train
    result = self._train()
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/function_runner.py", line 133, in _train
    result = self._status_reporter._get_and_clear_stat

...

(pid=14603) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=14604) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=14602) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=14605) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=14608) /Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
(pid=14608)   "Use dataset[()] instead.", H5pyDeprecationWarning)
(pid=14609) 

2019-03-13 14:53:57,442	ERROR trial_runner.py:426 -- Error processing event.
Traceback (most recent call last):
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/trial_runner.py", line 389, in _process_events
    result = self.trial_executor.fetch_result(trial)
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/ray_trial_executor.py", line 252, in fetch_result
    result = ray.get(trial_future[0])
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/worker.py", line 2288, in get
    raise value
ray.exceptions.RayTaskError: ray_worker (pid=14607, host=192.168.219.94)
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/trainable.py", line 150, in train
    result = self._train()
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/function_runner.py", line 133, in _train
    result = self._status_reporter._get_and_clear_stat

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/4 CPUs, 0/0 GPUs
Unknown memory usage. Please run `pip install psutil` (or ray[debug]) to resolve)
Result logdir: /Users/benmackenzie/ray_results/my_experiment
Number of trials: 24 ({'ERROR': 21, 'RUNNING': 3})
ERROR trials:
 - train_0_batch_size=4,num_fc_layers=1:	ERROR, 1 failures: /Users/benmackenzie/ray_results/my_experiment/train_0_batch_size=4,num_fc_layers=1_2019-03-13_14-53-017jhno16d/error_2019-03-13_14-53-12.txt
 - train_1_batch_size=8,num_fc_layers=1:	ERROR, 1 failures: /Users/benmackenzie/ray_results/my_experiment/train_1_batch_size=8,num_fc_layers=1_2019-03-13_14-53-01g2tvrvdc/error_2019-03-13_14-53-12.txt
 - train_2_batch_size=16,num_fc_layers=1:	ERROR, 1 failures: /Users/benmackenzie/ray_results/my_experiment/train_2_batch_size=16,num_fc_layers=1_2019-03-13_14-53-01h6117lvj/error_2019-03-13_14-53-12.txt
 - train_3_batch_size=32,num_fc_layers=1:	ERROR, 1 failures: /Users/benmackenzie/ray_results/my_exper

TuneError: ('Trials did not complete', [train_0_batch_size=4,num_fc_layers=1, train_1_batch_size=8,num_fc_layers=1, train_2_batch_size=16,num_fc_layers=1, train_3_batch_size=32,num_fc_layers=1, train_4_batch_size=64,num_fc_layers=1, train_5_batch_size=128,num_fc_layers=1, train_6_batch_size=4,num_fc_layers=2, train_7_batch_size=8,num_fc_layers=2, train_8_batch_size=16,num_fc_layers=2, train_9_batch_size=32,num_fc_layers=2, train_10_batch_size=64,num_fc_layers=2, train_11_batch_size=128,num_fc_layers=2, train_12_batch_size=4,num_fc_layers=3, train_13_batch_size=8,num_fc_layers=3, train_14_batch_size=16,num_fc_layers=3, train_15_batch_size=32,num_fc_layers=3, train_16_batch_size=64,num_fc_layers=3, train_17_batch_size=128,num_fc_layers=3, train_18_batch_size=4,num_fc_layers=4, train_19_batch_size=8,num_fc_layers=4, train_20_batch_size=16,num_fc_layers=4, train_21_batch_size=32,num_fc_layers=4, train_22_batch_size=64,num_fc_layers=4, train_23_batch_size=128,num_fc_layers=4])

(pid=14607) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=14609) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=14608) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=14606) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.


In [19]:
!ls


base_model.yaml       sequence_dataset.hdf5 transactions.csv
model.yaml            sequence_dataset.json transactions.hdf5
notebook1.ipynb       titanic.hdf5          transactions.json
notebook2.ipynb       titanic.json
results               titanic.yaml


In [27]:
!cat /Users/benmackenzie/ray_results/my_experiment/train_14_batch_size=16,num_fc_layers=3_2019-03-13_14-29-4342ycss03/error_2019-03-13_14-29-52.txt


Traceback (most recent call last):
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/trial_runner.py", line 389, in _process_events
    result = self.trial_executor.fetch_result(trial)
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/ray_trial_executor.py", line 252, in fetch_result
    result = ray.get(trial_future[0])
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/worker.py", line 2288, in get
    raise value
ray.exceptions.RayTaskError: ray_worker (pid=14151, host=192.168.219.94)
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/trainable.py", line 150, in train
    result = self._train()
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-packages/ray/tune/function_runner.py", line 133, in _train
    result = self._status_reporter._get_and_clear_status()
  File "/Users/benmackenzie/projects/ludwig/venv/lib/python3.6/site-pack